In [2]:
import pandas as pd

# CSV 파일 읽기
money = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\서울시_상권분석_서비스(추정매출-상권).csv')  # DataFrame 객체로 로드
human = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\서울시_상권분석_서비스(길단위인구-상권).csv')
map_sang = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\서울시_상권분석_서비스(영역-상권).csv')
coordinate = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\위도경도.csv')
market = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\점포.csv')

In [4]:
#필요한 컬럼만 선택하기
money = money[['기준_년분기_코드','상권_구분_코드_명','상권_코드','상권_코드_명','당월_매출_금액','남성_매출_금액','여성_매출_금액','서비스_업종_코드_명']]
human = human[['기준_년분기_코드','상권_코드','총_유동인구_수','남성_유동인구_수','여성_유동인구_수']]
map_sang = map_sang[['상권_코드','자치구_코드_명','행정동_코드_명']]
coordinate = coordinate[['상권_코드','경도1','위도1']]
market = market[['기준_년분기_코드','상권_코드','점포_수']]

In [5]:
# 추정매출과 길단위인구 조인 1step
join_1step = pd.merge(money, human, on=['기준_년분기_코드', '상권_코드'], how='inner')

In [6]:
# 1차에서 조인 되지 못한 상권코드
unmatched_from_df1 = money.merge(human, on=['기준_년분기_코드', '상권_코드'], how='left', indicator=True)
unmatched_code = unmatched_from_df1[unmatched_from_df1['_merge'] == 'left_only']

In [7]:
# 1step에서 조인 한것과 영역 데이터 조인(2차조인)
join_2step = pd.merge(join_1step, map_sang, on=['상권_코드'], how='inner')

In [8]:
#2차에서 조인 되지 못한 상권코드
unmatched_from_df1 = join_1step.merge(map_sang, on=['상권_코드'], how='left', indicator=True)
unmatched_code2 = unmatched_from_df1[unmatched_from_df1['_merge'] == 'left_only']

In [9]:
# '기준_년분기_코드' 값을 년월로 변환하는 함수 정의
def convert_to_date(code):
    year = int(str(code)[:4])  # 연도 추출
    quarter = int(str(code)[-1])  # 분기 추출
    month = (quarter - 1) * 3 + 1  # 분기에 따라 시작 월 계산
    return pd.Timestamp(year=year, month=month, day=1)
# 변환 적용
join_2step['기준_년분기_코드_날짜'] = join_2step['기준_년분기_코드'].apply(convert_to_date)

In [10]:
join_3step = join_2step[join_2step['기준_년분기_코드_날짜'] >= pd.Timestamp(year=2022, month=1, day=1)]


In [11]:
# 데이터 타입 최적화
coordinate['상권_코드'] = coordinate['상권_코드'].astype('category')
join_3step['상권_코드'] = join_3step['상권_코드'].astype('category')


C:\Users\m\AppData\Local\Temp\ipykernel_17972\1505405958.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  join_3step['상권_코드'] = join_3step['상권_코드'].astype('category')


In [12]:
# 2step에서 조인 한것과 영역 데이터 조인(3차조인)
join_4step = pd.merge(join_3step, coordinate, on=['상권_코드'], how='inner')

In [13]:
#3차에서 조인 되지 못한 상권코드
unmatched_from_df2 = join_3step.merge(coordinate, on=['상권_코드'], how='left', indicator=True)
unmatched_code3 = unmatched_from_df2[unmatched_from_df2['_merge'] == 'left_only']

In [14]:
# 중복 제거
join_4step = join_4step.drop_duplicates()

In [15]:
# 점포수 데이터 연결(4차연결)
join_final = pd.merge(join_4step, market, on=['기준_년분기_코드', '상권_코드'], how='inner')

In [17]:
# 4차에서 조인 되지 못한 상권코드
unmatched_from_df3 = join_4step.merge(market, on=['기준_년분기_코드', '상권_코드'], how='left', indicator=True)
unmatched_code4 = unmatched_from_df3[unmatched_from_df3['_merge'] == 'left_only']

In [18]:
unmatched_code4

,기준_년분기_코드,상권_구분_코드_명,상권_코드,상권_코드_명,당월_매출_금액,남성_매출_금액,여성_매출_금액,서비스_업종_코드_명,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,자치구_코드_명,행정동_코드_명,기준_년분기_코드_날짜,경도1,위도1,점포_수,_merge
2544,20222,골목상권,3110443,수색역 1번,16514626.0,9586128.0,6928498.0,분식전문점,91178,47311,43866,은평구,수색동,2022-04-01,126.934825,37.585158,NaN,left_only
4621,20222,골목상권,3110446,내를건너서숲으로도서관,4469435.0,2497775.0,1971660.0,예술학원,319631,169562,150070,은평구,신사2동,2022-04-01,126.941925,37.592102,NaN,left_only
8936,20222,골목상권,3111082,중앙보훈병원,5278580.0,2501827.0,2776753.0,미용실,140941,67131,73812,강동구,둔촌2동,2022-04-01,127.092343,37.534709,NaN,left_only
13024,20222,전통시장,3130062,보광시장,80539616.0,35096259.0,45443357.0,수산물판매,32190,15938,16251,용산구,보광동,2022-04-01,127.000007,37.529058,NaN,left_only
16644,20222,전통시장,3130009,마전교지하쇼핑센터(구 한일상가),238548259.0,151831400.0,86716859.0,운동/경기용품,71488,37553,33935,종로구,종로5·6가동,2022-04-01,127.001890,37.571288,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3347207,20234,골목상권,3110953,학동근린공원,83419952.0,1340742.0,77870151.0,커피-음료,427812,208423,219389,강남구,논현1동,2023-10-01,127.016996,37.517353,NaN,left_only
3351146,20234,골목상권,3110462,갈현건영아파트,153642029.0,109688633.0,43953396.0,편의점,506850,215010,291840,은평구,갈현1동,2023-10-01,126.947489,37.626849,NaN,left_only
3351927,20234,골목상권,3110784,신대림초등학교,159571205.0,110414933.0,42534114.0,중식음식점,182503,90803,91701,영등포구,대림1동,2023-10-01,126.939858,37.493431,NaN,left_only
3356133,20234,전통시장,3130011,충신시장,23452404.0,17179132.0,1740236.0,한식음식점,234946,119593,115353,종로구,종로5·6가동,2023-10-01,127.003511,37.575411,NaN,left_only


In [19]:
#최종 데이터 저장
join_final.to_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\종합_대시보드데이터.csv', index=False, encoding='utf-8-sig')
unmatched_code.to_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\유동인구없는_상권코드.csv', index=False, encoding='utf-8-sig')
unmatched_code4.to_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\점포수없는_상권코드.csv', index=False, encoding='utf-8-sig')